# Plan, strategize, and automate data backup on AWS with AI Agents on Amazon Bedrock

[AWS Backup](https://aws.amazon.com/backup/) is a fully managed backup service centralizing and automating the backup of data across AWS services. AWS Backup provides an orchestration layer that integrates Amazon CloudWatch, AWS CloudTrail, AWS Identity and Access Management (IAM), AWS Organizations, and other services. This centralized, AWS Cloud native solution provides global backup capabilities that can help you achieve your disaster recovery and compliance requirements. Using AWS Backup, you can centrally configure backup policies and monitor backup activity for AWS resources.

Some AWS services like [Amazon EC2](https://aws.amazon.com/ec2/), [Amazon S3](https://aws.amazon.com/s3/), [Amazon RDS](https://aws.amazon.com/rds/) etc., will let you configure your data backups natively in those services i.e. outside of AWS Backup.

In all these cases, when creating a data backup strategy, you will need to understand the concepts, best practices, and architecture principles behind data backup and recovery. You would want to make sure all your resources are covered. While these can be challenging, configuring backups either within AWS Backup or natively in other services for your backup objectives requires knowledge of AWS Backup or other service APIs, and how to invoke one or more of them in a sequence based on your requirements. AI agents can help with all these.

This notebook will walk you through the process of using [Amazon Bedrock Agents](https://aws.amazon.com/bedrock/agents/) along with the [Large Language Models (LLMs)](https://docs.aws.amazon.com/bedrock/latest/userguide/models-supported.html) on Amazon Bedrock to provide an easy-to-use natural language interface to strategize, plan, and execute your data backup objectives using [AWS Backup](https://aws.amazon.com/backup/), [Amazon EC2](https://aws.amazon.com/ec2/), [Amazon S3](https://aws.amazon.com/s3/), and [Amazon RDS](https://aws.amazon.com/rds/).

[Amazon Bedrock](https://aws.amazon.com/bedrock/) is a fully managed service that offers a choice of high-performing Foundation Models (FMs) from leading AI companies accessible through a single API, along with a broad set of capabilities you need to build generative AI applications, simplifying development while maintaining privacy and security.

[Large Language Models (LLMs)](https://en.wikipedia.org/wiki/Large_language_model) are a type of Foundation Model that can take natural langauge as input, with the ability to process and understand it, and produce natural language as the output. LLMs can also can perform tasks like classification, summarization, simplification, entity recognition, etc.

In the process of working through this notebook, you will learn how to setup and interact with Amazon Bedrock Agents, Knowledge Bases, setup the client environment, configure security permissions, and use prompt templates. You will also learn how to use Amazon Bedrock's [Converse API](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_Converse.html) to consistently interact with multiple supported LLMs.

<div class="alert alert-block alert-info">
<b>Note:</b>
    <ul>
        <li>This notebook should only be run from within an <a href="https://docs.aws.amazon.com/sagemaker/latest/dg/nbi.html">Amazon SageMaker Notebook instance</a> or within an <a href="https://docs.aws.amazon.com/sagemaker/latest/dg/studio-updated.html">Amazon SageMaker Studio Notebook</a>.</li>
        <li>At the time of writing this notebook, Amazon Bedrock was only available in <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/bedrock-regions.html">these supported AWS Regions</a>. If you are running this notebook from any other AWS Region, then you have to change the Amazon Bedrock Agent client's region and/or endpoint URL parameters to one of those supported AWS Regions that has <b>Anthropic Claude 3.5 Haiku v1, Anthropic Claude 3.5 Sonnet v2 / 3.7 Sonnet v1, and Amazon Titan Text Embeddings v2</b>. If available, you can use <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/cross-region-inference.html">cross-region inference</a>. Follow the guidance in the <i>Organize imports</i> section of this notebook.</li>
        <li>This notebook is recommended to be run with a minimum instance size of <i>ml.t3.medium</i> and
            <ul>
                <li>With <i>Amazon Linux 2, Jupyter Lab 4</i> as the platform identifier on an Amazon SageMaker Notebook instance.</li>
                <li> (or)
                <li>With <i>Data Science 3.0</i> as the image on an Amazon SageMaker Studio Notebook.</li>
            <ul>
        </li>
        <li>At the time of this writing, the most relevant latest version of the Kernel for running this notebook,
            <ul>
                <li>On an Amazon SageMaker Notebook instance was <i>conda_python3</i></li>
                <li>On an Amazon SageMaker Studio Notebook was <i>Python 3</i></li>
            </ul>
        </li>
    </ul>
</div>

## Architecture

<center><img src="images/flow_architecture.png"/></center>

**Overall Data flow:**

0. Ingest the information about architecture, and best practices:
    1. **(Step 0a):** Download the information from HTTP/S URLs or start the ingestion of the files from an Amazon S3 bucket.
    2. **(Step 0b):** Ingest into a Knowledge Base (KB) built on Amazon Bedrock Knowledge Bases.
    3. **(Step 0c):** During the ingestion process, the data is chunked using a fixed-size chunking strategy, and vectorized by calling an Embeddings model (Amazon Titan Text Embeddings v2).
    4. **(Step 0d):** The vectorized data is stored in the KB along with the required metadata like the source information. This KB uses an Amazon OpenSearch Serverless (AOSS) based vector store.
1. **(Step 1):** The user sends a message to the chat assistant with what they want to accomplish.
2. **(Step 2):** The chat assistant provides the UI for the user. It receives the user's message and sends it to the supervisor agent (built on Amazon Bedrock Agents).
3. Supervisor agent (built on Amazon Bedrock Agents):
    1. **(Step 3a):** The supervisor agent uses LLM 2 (Anthropic Claude 3.5 Sonnet v2 / 3.7 Sonnet v1) to determine the appropriate sub-supervisor agent based on the service involved (AWS Backup / Amazon EC2 / Amazon S3 / Amazon RDS) or the Knowledge agent to process the user's request. Sometimes multiple agents will be invoked based on the user's request. In such cases, the responses from various sub-agents will be consolidated.
    2. **(Step 3b):** The supervisor agent routes the request to the service-specific sub-supervisor agent as determined in the previous step.
    3. **(Step 3c):** The supervisor agent routes the request to the Knowledge agent as determined in the previous step.
    4. The response or re-prompt from the sub-supervisor agent is sent back to the user through the chat assistant.
4. Sub-supervisor agent (built on Amazon Bedrock Agents):
    1. **(Step 4a):** The sub-supervisor agent, which is specific to the service (AWS Backup / Amazon EC2 / Amazon S3 / Amazon RDS), uses LLM 1 (Anthropic Claude 3.5 Haiku v1) to determine whether the user's request pertains to a planning or execution operation and routes it to the appropriate sub-agent. **Note:** Except for AWS Backup, the planner and executor sub-agents for other services are contained within the sub-supervisor agent.
    2. **(Step 4b):** Planner sub-agent (built on Amazon Bedrock Agents):
        1. **(Step P1):** This agent uses LLM 2 (Anthropic Claude 3.5 Sonnet v2 / 3.7 Sonnet v1) to break down the user request into tasks. It will also determine if an AWS Backup / Amazon EC2 / Amazon S3 / Amazon RDS operation like describe, get, or list needs to be performed or not.
        2. **(Step P2):** If a describe, get, or list operation is required, the agent will invoke an action group which is associated with an AWS Lambda function. This function will receive the original message from the user, along with the boto3 JSON API for the appropriate operation. *Refer to the next section for the data flow within the Lambda function.*
        3. Finally, it will consolidate all this information and send it back to the sub-supervisor agent. If any additional info is required, this agent will re-prompt the user through the sub-supervisor agent.
    3. **(Step 4c):** Executor sub-agent (built on Amazon Bedrock Agents):
        1. **(Step E1):** This agent uses LLM 2 (Anthropic Claude 3.5 Sonnet v2 / 3.7 Sonnet v1) to break down the user request into tasks. It will also determine if an AWS Backup / Amazon EC2 / Amazon S3 / Amazon RDS operation like create, update, cancel, or delete needs to be performed or not.
        2. **(Step E2):** If a create, update, cancel, or delete operation is required, the agent will invoke an action group which is associated with an AWS Lambda function. This function will receive the original message from the user, along with the boto3 JSON API for the appropriate AWS Backup / Amazon EC2 / Amazon S3 / Amazon RDS operation. *Refer to the next section for the data flow within the Lambda function.*
        3. Finally, it will consolidate all this information and send it back to the sub-supervisor agent. If any additional info is required, this agent will re-prompt the user through the sub-supervisor agent.
5. Knowledge agent (built on Amazon Bedrock Agents):
    1. **(Step K1):** The Knowledge agent uses LLM 2 (Anthropic Claude 3.5 Sonnet v2 / 3.7 Sonnet v1) to determine if a search and retrieve operation against the configured Knowledge Base (KB) is required or not.
    2. **(Step K2):** If a KB lookup is required, it is performed. This KB will contain all the required information about data backup and restore, disaster recovery (DR), data protection, and data compliance.
    3. Finally, it will process the information retrieved from the KB and send it back to the supervisor agent. If any additional info is required, this agent will re-prompt the user through the supervisor agent.

**Lambda function data flow:**

1. **(Step L1):** The Lambda function, which is specific to the service (AWS Backup / Amazon EC2 / Amazon S3 / Amazon RDS), will perform pre-processing, and will attempt to improve the boto3 JSON API (generated by the agent) by invoking LLM 2 (Anthropic Claude 3.5 Sonnet v2 / 3.7 Sonnet v1). It will then use this JSON to invoke the AWS Backup API **(Step L3)**.
2. **(Step L2):** If the API call fails, then, the function will invoke LLM 2 to fix the JSON based on the error message. It will then invoke the AWS Backup API **(Step L3)** with the improved JSON.
3. If this call succeeds, then the result will be returned to the sub-agent. If it fails, then, the failure message will be returned to the sub-agent.

**Supported AWS Backup operations:**
(Note: Mandatory parameters are mentioned in brackets. All other parameters are either auto-generated or optional.)
1. Create backup vault
2. Create backup plan
3. Update backup plan (using plan id and name)
4. Create backup selection (using plan id)
5. Create legal hold
6. List backup vaults (all or using vault-type/tags)
7. List protected resources (all or using resource name or backup vault name)
8. List backup jobs (all or using resource type)
9. List backup plans (all or using tags)
10. List backup selections (all or using backup plan id or selection name)
11. List legal holds (all or using tags)
12. Get backup vault (using vault id/name/arn)
13. Get backup plan (using plan id/name)
14. Get backup selection (using plan id/name and selection id/name)
15. Get legal hold (using id/ARN)
16. Delete backup vault (using vault id/name/arn)
17. Delete backup plan (using plan id/name)
18. Delete backup selection (using plan id/name and selection id)
19. Cancel legal hold (using id/ARN)
20. List all recovery points (using backup vault name or resource ARN or legal hold id or resource type)

**Supported Amazon EC2 operations:**
(Note: Mandatory parameters are mentioned in brackets. All other parameters are either auto-generated or optional.)
1. List all instances (all or using id/name/tags)
2. List all volumes (all or using volume id/name/tags or instance id/name/tags)
3. List all snapshots (all or using snapshot id/name/tags or volume id/name/tags or instance id/name/tags)
4. Create a snapshot for a volume (using volume id)
5. Delete a snapshot (using snapshot id)

**Supported Amazon S3 operations:**
(Note: Mandatory parameters are mentioned in brackets. All other parameters are either auto-generated or optional.)
1. List all buckets (all or using region name and/or tags)
2. Get bucket replication info
3. Get bucket versioning info
4. Get bucket lifecycle config

**Supported Amazon RDS operations:**
(Note: Mandatory parameters are mentioned in brackets. All other parameters are either auto-generated or optional.)
1. List all db clusters (all or using id/name/tags)
2. List all db instances (all or using id/name/tags)
3. List all db cluster automated backups
4. List all db instance automated backups
5. Start db instance automated backups replication
6. Stop db instance automated backups replication
7. Delete db cluster automated backup
8. Delete db instance automated backup

<div class="alert alert-block alert-warning">  
    <b>Note:</b> In all the supported operations specified above in this notebook, list and search operations are restricted to the first 100 items. When you perform these operations, there is a possibility that the item you are looking for may be beyond the list of those 100 processed items. So you may not see it in the results. For example, if you run the operation to list all Amazon S3 buckets in a region and if there are more than 100 buckets in that region, then, only the first 100 buckets will be retrieved and processed.
</div>

## Running this notebook

**Table of Contents:**

1. [Complete prerequisites](#Complete%20prerequisites)

    1. [Check and configure access to the Internet](#Check%20and%20configure%20access%20to%20the%20Internet)

    2. [Install required software libraries](#Install%20required%20software%20libraries)
    
    3. [Configure logging](#Configure%20logging)
        
        1. [System logs (Optional)](#Configure%20system%20logs%20(Optional))
        
        2. [Application logs](#Configure%20application%20logs)
    
    4. [Organize imports](#Organize%20imports)
    
    5. [Set AWS Region and boto3 config](#Set%20AWS%20Region%20and%20boto3%20config)
    
    6. [Enable model access in Amazon Bedrock](#Enable%20model%20access%20in%20Amazon%20Bedrock)
    
    7. [Create common objects](#Create%20common%20objects)
    
    8. [Check and configure security permissions](#Check%20and%20configure%20security%20permissions)
  
    9. [Get Bedrock Agent details](#Get%20Bedrock%20Agent%20details)

 2. [Ingest into Knowledge Base](#Ingest%20into%20Knowledge%20Base)

    1. [Ingest from HTTP/S data source](#Ingest%20from%20HTTP/S%20data%20source)
       
    2. [Ingest from S3 data source](#Ingest%20from%20S3%20data%20source)
 
 4. [Perform backup actions](#Perform%20backup%20actions)

    1. [(Option 1) For chat and single-prompt](#For%20chat%20and%20single-prompt)
   
    2. [(Option 2) For single-prompt only](#For%20single-prompt%20only)
 
 5. [Cleanup](#Cleanup)
 
 6. [Conclusion](#Conclusion)
 
 7. [Frequently Asked Questions (FAQs)](#FAQs)

##  1. Complete prerequisites <a id ='Complete%20prerequisites'> </a>

Check and complete the prerequisites.

###  A. Check and configure access to the Internet <a id ='Check%20and%20configure%20access%20to%20the%20Internet'> </a>
This notebook requires outbound access to the Internet to download the required software updates, to download the required files, and interact with the required AWS services.  You can either provide direct Internet access (default) or provide Internet access through an [Amazon VPC](https://aws.amazon.com/vpc/).  For more information on this, refer [here](https://docs.aws.amazon.com/sagemaker/latest/dg/appendix-notebook-and-internet-access.html).

### B. Install required software libraries <a id ='Install%20required%20software%20libraries'> </a>
This notebook requires the following libraries:
* [SageMaker Python SDK version 2.x](https://sagemaker.readthedocs.io/en/stable/v2.html)
* [Python 3.10.x](https://www.python.org/downloads/release/python-3100/)
* [Boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html)

Run the following cell to install the required libraries.

<div class="alert alert-block alert-warning">  
    <b>Note:</b> At the end of the installation, the Kernel will be forcefully restarted immediately. Please wait 10 seconds for the kernel to come back before running the next cell.
</div>

In [ ]:
!pip install boto3==1.40.0
!pip install requests==2.32.4
!pip install sagemaker==2.249.0

import IPython

IPython.Application.instance().kernel.do_shutdown(True)

### C. Configure logging <a id ='Configure%20logging'> </a>

####  a. System logs (Optional) <a id='Configure%20system%20logs%20(Optional)'></a>

System logs refers to the logs generated by the notebook's interactions with the underlying notebook instance. Some examples of these are the logs generated when loading or saving the notebook.

These logs are automatically setup when the notebook instance is launched.

These logs can be accessed through the [Amazon CloudWatch Logs](https://docs.aws.amazon.com/AmazonCloudWatch/latest/logs/WhatIsCloudWatchLogs.html) console in the same AWS Region where this notebook is running.
* When running this notebook in an Amazon SageMaker Notebook instance, navigate to the following location,
    * <i>CloudWatch > Log groups > /aws/sagemaker/NotebookInstances > {notebook-instance-name}/jupyter.log</i>
* When running this notebook in an Amazon SageMaker Studio Notebook, navigate to the following locations,
    * <i>CloudWatch > Log groups > /aws/sagemaker/studio > {sagmaker-domain-name}/{user-name}/KernelGateway/{notebook-instance-name}</i>
    * <i>CloudWatch > Log groups > /aws/sagemaker/studio > {sagmaker-domain-name}/{user-name}/JupyterServer/default</i>

If you want to find out the name of the underlying instance where this notebook is running, uncomment the following code cell and run it.

In [ ]:
'''
import json

notebook_name = ''
resource_metadata_path = '/opt/ml/metadata/resource-metadata.json'
with open(resource_metadata_path, 'r') as metadata:
    notebook_name = (json.load(metadata))['ResourceName']
print("Notebook instance name: '{}'".format(notebook_name))
'''

####  b. Application logs <a id='Configure%20application%20logs'></a>

Application logs refers to the logs generated by running the various code cells in this notebook. To set this up, instantiate the [Python logging service](https://docs.python.org/3/library/logging.html) by running the following cell. You can configure the default log level and format as required.

By default, this notebook will only print the logs to the corresponding cell's output console.

In [ ]:
import logging
import os

# Set the logging level and format
log_level = logging.INFO
log_format = '%(asctime)s - %(levelname)s - %(message)s'
logging.basicConfig(level=log_level, format=log_format)

# Save these in the environment variables for use in the helper scripts
os.environ['LOG_LEVEL'] = str(log_level)
os.environ['LOG_FORMAT'] = log_format

###  D. Organize imports <a id ='Organize%20imports'> </a>

Organize all the library and module imports for later use.

In [ ]:
import boto3
import os
import requests
import sagemaker
import sys
from botocore.config import Config

# Import the helper functions from the 'scripts' folder
sys.path.append(os.path.join(os.getcwd(), "scripts"))
#logging.info("Updated sys.path: {}".format(sys.path))
from helper_functions import *

Print the installed versions of some of the important libraries.

In [ ]:
logging.info("Python version : {}".format(sys.version))
logging.info("Boto3 version : {}".format(boto3.__version__))
logging.info("Requests version : {}".format(requests.__version__))
logging.info("SageMaker Python SDK version : {}".format(sagemaker.__version__))

###  E. Set AWS Region and boto3 config <a id ='Set%20AWS%20Region%20and%20boto3%20config'> </a>

Get the current AWS Region (where this notebook is running) and the SageMaker Session. These will be used to initialize some of the clients to AWS services using the boto3 APIs.

<div class="alert alert-block alert-info">  
<b>Note:</b> All the AWS services used by this notebook except Amazon Bedrock will use the current AWS Region. For Bedrock, follow the guidance in the next cell.
</div>

<div class="alert alert-block alert-warning">  
<b>Note:</b> At the time of writing this notebook, Amazon Bedrock was only available in <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/bedrock-regions.html">these supported AWS Regions</a>. If you are running this notebook from any other AWS Region, then you have to change the Amazon Bedrock Agent client's region and/or endpoint URL parameters to one of those supported AWS Regions that has Anthropic Claude 3.5 Haiku v1, Anthropic Claude 3.5 Sonnet v2 / 3.7 Sonnet v1, and Amazon Titan Text Embeddings v2. In order to do this, this notebook will use the value specified in the environment variable named <mark>AMAZON_BEDROCK_REGION</mark>. If this is not specified, then the notebook will default to <mark>us-west-2 (Oregon)</mark> for Amazon Bedrock. If available, you can use <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/cross-region-inference.html">cross-region inference</a>.
</div>



In [ ]:
# Get the AWS Region, SageMaker Session and IAM Role references
my_session = boto3.session.Session()
logging.info("SageMaker Session: {}".format(my_session))
my_iam_role = sagemaker.get_execution_role()
logging.info("Notebook IAM Role: {}".format(my_iam_role))
my_region = my_session.region_name
logging.info("Current AWS Region: {}".format(my_region))

# Explicity set the AWS Region for Amazon Bedrock clients
AMAZON_BEDROCK_DEFAULT_REGION = "us-west-2"
br_region = os.environ.get('AMAZON_BEDROCK_REGION')
if br_region is None:
    br_region = AMAZON_BEDROCK_DEFAULT_REGION
elif len(br_region) == 0:
    br_region = AMAZON_BEDROCK_DEFAULT_REGION
logging.info("AWS Region for Amazon Bedrock: {}".format(br_region))

Set the timeout and retry configurations that will be applied to all the boto3 clients used in this notebook.

In [ ]:
# Increase the standard time out limits in the boto3 client from 1 minute to 3 minutes
# and set the retry limits
my_boto3_config = Config(
    connect_timeout = (60 * 3),
    read_timeout = (60 * 3),
    retries = {
        'max_attempts': 10,
        'mode': 'standard'
    }
)

###  F. Enable model access in Amazon Bedrock <a id ='Enable%20model%20access%20in%20Amazon%20Bedrock'> </a>

<div class="alert alert-block alert-danger">
    <b>Note:</b> Before proceeding further with this notebook, you must enable access to the required models on Amazon Bedrock by following the instructions <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html">here</a>. Otherwise, you will get an authorization error.
</div>

<div class="alert alert-block alert-warning">  
<b>Note:</b> You will have to do this manually after reading the End User License Agreement (EULA) for each of the models that you want to enable. Unless you explicitly disable it, this is a one-time setup for each model in an AWS Region in an AWS account.
</div>

Run the following cell to print the Amazon Bedrock model access page URL for the AWS Region that was selected earlier.

In [ ]:
# Print the Amazon Bedrock model access page URL
print("https://{}.console.aws.amazon.com/bedrock/home?region={}#/modelaccess".format(br_region, br_region))

###  G. Create common objects <a id='Create%20common%20objects'></a>

To begin with, create the boto3 clients.

In [ ]:
# Create the Agents for Amazon Bedrock client
bedrock_agt_client = boto3.client("bedrock-agent", region_name = br_region, config = my_boto3_config)

# Create the Agents for Amazon Bedrock runtime client
bedrock_agt_rt_client = boto3.client("bedrock-agent-runtime", region_name = br_region, config = my_boto3_config)

###  H. Check and configure security permissions <a id ='Check%20and%20configure%20security%20permissions'> </a>
This notebook uses the IAM role attached to the underlying notebook instance.  To view the name of this role, run the following cell. This IAM role should have the following permissions,
1. Full access to all the Amazon Bedrock Agents used in this notebook.
2. Full access to the Amazon Bedrock Knowledge Base used in this notebook.
3. Access to write to the Amazon S3 bucket from where it will be ingested into the Knowledge Base (KB).
4. Access to write to Amazon CloudWatch Logs.

The Amazon Bedrock Knowledge Base associated with the Bedrock Agents used in this notebook, requires the following permissions,
1. Access to read and write to the Amazon OpenSearch Serverless Collection's index used by the Knowledge Base (KB).
2. Access to read from the Amazon S3 bucket that will contain the files to ingestion into the KB.
3. Access to write to Amazon CloudWatch Logs.

The Bedrock Agents that this notebook will invoke, require the following permissions,
1. Access to the invoke the associated AWS Lambda functions.
2. Access to invoke the configured sub-agents.
3. Access to invoke Large Language Models (LLMs) on Amazon Bedrock.
4. Access to retrieve from the Knowledge Base configured above.
5. Access to write to Amazon CloudWatch Logs.

The AWS Lambda function associated with each of the above Bedrock Agents requires the following permissions,
1. Access to read and write to one of these services based on the agent - AWS Backup or Amazon EC2 or Amazon S3 or Amazon RDS.
2. Access to invoke Large Language Models (LLMs) on Amazon Bedrock.
3. Access to write to Amazon CloudWatch Logs.

<div class="alert alert-block alert-info">
<b>Note:</b>  If you are running this notebook as part of a workshop session, by default, all these permissions will be setup.
</div>

Run the following cell to print the details of the IAM role attached to the underlying notebook instance.

In [ ]:
# Print the IAM role ARN and console URL
logging.info("This notebook's IAM role is '{}'".format(my_iam_role))
arn_parts = my_iam_role.split('/')
logging.info("Details of this IAM role are available at https://{}.console.aws.amazon.com/iamv2/home?region={}#/roles/details/{}?section=permissions"
             .format(my_region, my_region, arn_parts[len(arn_parts) - 1]))

###  I. Get Bedrock Agent details <a id='Get%20Bedrock%20Agent%20details'></a>

<center><img src="images/agents_architecture.png"/></center>

<div class="alert alert-block alert-info">
<p><b>Note:</b> For the purpose of running this notebook, all the Bedrock Agents specified in the above diagram along with their agent aliases must be created in the same AWS Region as Amazon Bedrock that was configured in Step 1E of this notebook and must be in 'PREPARED' status.</p>
<p>
Although not mandatory, it is strongly recommended to use Anthropic Claude 3.5 Haiku v1 / Anthropic Claude 3.5 Sonnet v2 / 3.7 Sonnet v1 as the LLMs for various agents as specified in steps earlier. If you use any other LLM, make sure the model access is enabled by following the instructions mentioned in prior steps.
</p>
<p>If you are running this notebook as part of a workshop session, by default, all these agents along with their aliases that meet all these requirements will be pre-created and ready to use.</p>
</div>

<div class="alert alert-block alert-danger">
    <b>Note:</b> If you are running this notebook outside of a workshop session, then, you must create these Bedrock Agents along with their aliases as specified above. Otherwise, this notebook will fail. You can follow the procedure described <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/agents-create.html">here</a> to create a Bedrock Agent and the procedure described <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/agents-alias-manage.html">here</a> to create an agent alias.
</div>

Specify the name of the supervisor agent in the cell below. 

In [ ]:
br_supervisor_agent_name = 'backup-assistant-supervisor'

Run the following cell to verify if the specified supervisor agent meets all the requirements mentioned above. If they meet all the requirements, then,
* The agent's version and alias id, if specified, will be used. If not, these two values will be retrieved from the supervisor agent's configuration.
* All the Knowledge Bases (KBs) associated with the supervisor and all it's collaborator agents will be retrieved.
* For every KB that is in ENABLED status, the first custom and S3 data sources that are in AVAILABLE status will be retrieved and used.
* All the AWS Lambda functions associated with the supervisor and all it's collaborator agents will be retrieved.

All these will be required later in the notebook.

In [ ]:
br_supervisor_agent_id = ''
br_supervisor_agent_version = ''
br_supervisor_agent_alias_id = ''
kb_id = ''
kb_custom_ds_id = ''
kb_s3_ds_id = ''
lambda_function_name = ''

# Note: 'does_br_agent_meet_requirements' available through ./scripts/helper_functions.py
meets_requirements, br_supervisor_agent_id, br_supervisor_agent_version = does_br_agent_meet_requirements(bedrock_agt_client, br_supervisor_agent_name)
if meets_requirements:
    # Note: 'get_br_agent_alias_that_meets_requirements' available through ./scripts/helper_functions.py
    br_supervisor_agent_alias_id = get_br_agent_alias_that_meets_requirements(bedrock_agt_client, br_supervisor_agent_id, br_supervisor_agent_alias_id)
    # Note: 'get_kbs' available through ./scripts/helper_functions.py
    kbs = get_kbs(bedrock_agt_client, br_supervisor_agent_id, br_supervisor_agent_version)
    # Loop through the KBs
    for kb in kbs:
        # Get the id of the KB that is enabled
        if kb['state'] == 'ENABLED':
            kb_id = kb['id']
            # Note: 'get_kb_data_sources' available through ./scripts/helper_functions.py
            data_sources = get_kb_data_sources(bedrock_agt_client, kb_id)
            # Loop through the data sources
            for data_source in data_sources:
                if data_source['status'] == 'AVAILABLE':
                    if data_source['type'] == 'CUSTOM':
                        kb_custom_ds_id = data_source['id']
                    elif data_source['type'] == 'S3':
                        kb_s3_ds_id = data_source['id']
    logging.info("Knowledge Base id = '{}'; Custom data source id = '{}'; S3 data source id = '{}'".format(kb_id, kb_custom_ds_id, kb_s3_ds_id))
    # Note: 'get_lambda_functions' available through ./scripts/helper_functions.py
    lambda_functions = get_lambda_functions(bedrock_agt_client, br_supervisor_agent_id, br_supervisor_agent_version)
    # Loop through the lambda_functions
    for lambda_function in lambda_functions:
        lambda_function_name = lambda_function
        logging.info("Lambda function name = '{}'".format(lambda_function_name))
else:
    logging.error("The specified supervisor agent does not meet all requirements.")

## 2. Ingest into Knowledge Base <a id='Ingest%20into%20Knowledge%20Base'></a>

Run the following cells to ingest custom content about backup best practices, recommendations, and architecture. This content can either HTTP/S URLs or in the form of files stored in an Amazon S3 bucket. This content will provide the [Retrieval Augmented Generation (RAG)](https://aws.amazon.com/what-is/retrieval-augmented-generation/) data that will stored in the Knowledge Base (KB) and used in the prompts sent to the LLMs by the agents.

**Note:** If this custom content was publicly available at the time when the model was trained, it is most likely that the model was already trained with this content.

####  a. Ingest from HTTP/S data source <a id='Ingest%20from%20HTTP/S%20data%20source'></a>

Running the following cell will ingest data from HTTP/S URLs.

In [ ]:
# Specify the contents to ingest from
content_http_urls = []
#content_http_urls.append('https://aws.amazon.com/blogs/security/top-10-security-best-practices-for-securing-backups-in-aws/')
content_http_urls.append('https://docs.aws.amazon.com/organizations/latest/userguide/orgs_manage_policies_backup_best-practices.html')

# Loop through the list of contents
for content_http_url in content_http_urls:
    # Note: 'ingest_into_kb' available through ./scripts/helper_functions.py
    ingestion_status, ingestion_status_reason = ingest_into_kb(bedrock_agt_client, kb_id, kb_custom_ds_id, content_http_url)
    logging.info("Content URL :: {}; Ingestion Status :: {}, Ingestion Status Reason :: {}".format(content_http_url,
                                                                                                   ingestion_status,
                                                                                                   ingestion_status_reason))

####  b. Ingest from S3 data source <a id='Ingest%20from%20S3%20data%20source'></a>

Running the following cell will ingest data from an Amazon S3 bucket by executing the following steps:
1. Retrieve the S3 bucket details from the S3 data source associated with the KB.
2. (Optional) Download the files from HTTP/S URLs to the specified local directory.
3. Upload the files from the specified local directory to this S3 bucket.
4. Trigger the [sync operation](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base-ingest.html) on that Knowledge Base to load the file(s) from that data source (S3) to the vector index configured for that Knowledge Base (KB). In the process of loading, the data will be chunked using a fixed-size chunking strategy, and converted to vectors using the Embeddings Model specified for that KB. Check the status of the ingestion every 5 seconds until failed or complete.

In [ ]:
# Retrieve the S3 bucket ARN and name from the S3 data source of the KB
s3_bucket_arn, s3_bucket_name = get_bucket_from_kb_s3_data_source(bedrock_agt_client, kb_id, kb_s3_ds_id)

# Specify the S3 bucket prefix
s3_key_prefix = "rag_data"

# Specify the data directory and create it
rag_data_dir = os.path.join(os.getcwd(), "rag_data")
os.makedirs(rag_data_dir, exist_ok = True)

<div class="alert alert-block alert-info">
    <b>Note:</b> With this option to ingest from a S3 data source, the files are expected to be in the specified local directory. You can upload the files to this directory yourself or download them from HTTP/S URLs, although for this, you are better off going with the first option.
</div>


(Optional) Run the following cell to download files from HTTP/S URLs to the specified local directory.

In [ ]:
# Specify the contents to download from
content_http_urls = []
content_http_urls.append('https://docs.aws.amazon.com/pdfs/prescriptive-guidance/latest/backup-recovery/backup-recovery.pdf')
content_http_urls.append('https://esentire-dot-com-assets.s3.amazonaws.com/assets/resourcefiles/Checklist_DORA-Compliance-Checklist_Sep2024.pdf')

# Loop through the list of contents
for content_http_url in content_http_urls:
    # Note: 'download_file' available through ./scripts/helper_functions.py
    download_file(content_http_url, rag_data_dir)

Run the following cell to upload the file(s) to S3 and start the ingestion to the KB.

In [ ]:
# Note: 'upload_to_s3' available through ./scripts/helper_functions.py
upload_to_s3(rag_data_dir, s3_bucket_name, s3_key_prefix)

# Note: 'sync_to_kb' available through ./scripts/helper_functions.py
sync_to_kb(bedrock_agt_client, kb_s3_ds_id, kb_id, 'Sync PDF from S3 to vector index')

## 3. Perform backup actions <a id='Perform%20backup%20actions'></a>

Perform the various AWS Backup actions by invoking the `supervisor agent` configured in prior steps. You can work with this agent through the following mechanisms,

1. **Chat:**
These are back-and-forth conversations. In this scenario, the agent will remember all prior interactions in that session. This will provide the full context for the agent to answer your questions, ask follow-up questions or perform the required operations. A session will end,
   - When it is idle for some time, typically 10 minutes. This is configurable.
   - When you explicitly end the current session.
2. **Single-prompt:**
These are single query conversations. In the scenario, the agent will only remember this one query. Every invocation of the agent will be a new session. With this type of interaction, you have to provide all the details to accomplish a task within a single prompt. Otherwise, the agent will re-prompt and that will not be useful because the session will end after every interaction. For example, in single-prompt interactions, note the following,

    - Explicitly specify the AWS Region where you want the backup operations to run.
    - Don't specify any delete operations as they will require an additional confirmation.
    - Will be useful to add "Assume other values as required." at the end of the prompt.

**Note:** For examples of queries, refer to the code cell two sections below.

###  A. (Option 1): For chat and single-prompt <a id='For%20chat%20and%20single-prompt'></a>

For working with the supervisor agent using either chat or single-prompt interactions, starting new sessions, and a rich UI experience, go to the link printed by running the cell below. This will take you to the supervisor agent's test widget, displayed on the right hand side of the window.

In [ ]:
print("https://{}.console.aws.amazon.com/bedrock/home?region={}#/agents/{}/alias/{}".format(br_region, br_region, br_supervisor_agent_id, br_supervisor_agent_alias_id))

###  B. (Option 2): For single-prompt only <a id='For%20single-prompt%20only'></a>

For working with the supervisor agent using single-prompt interactions, specify the required query in the `query` variable in the below cell and run it.

In [ ]:
# Configure the query

#### Examples of Knowledge Agent queries
#query = 'Why should I use AWS as a data-protection platform?'
#query = 'How does the AWS Backup service work?'
#query = 'What are the top 2 best practices for securing backups in AWS?'
#query = 'Tell me about Article 11 in the DORA Compliance Checklist.'

#### Examples of planner queries

## AWS Backup queries
#query = 'How am I doing with my backups in us-west-2?'
#query = 'List all backup vaults in us-west-2.'
#query = 'List all backup plans in us-west-2.'
#query = 'In us-west-2, list all backup selections associated with backup plan named "TEST".'
#query = 'In us-east-1, list all resources protected by AWS Backup.'
#query = 'In us-east-1, list all protected resources in backup vault named "TEST".'
#query = 'In us-east-1, is there a backup vault named "TEST"?'
#query = 'In us-east-1, is there a backup plan with tag "Name" and value "Value"?'
#query = 'In us-east-1, is there a backup plan with tag "Key" and value "Value"?'
#query = 'Get the legal hold with ARN "ARN".'

## Amazon EC2 queries
#query = 'In us-east-1, list all snapshots for volume with id "value".'
#query = 'In us-east-1, list all snapshots for volume with name "name".'
#query = 'In us-east-1, get the snapshots associated with the instances with tag named "os:Type" and values "Windows" and "AmazonLinux2023".'
#query = 'In us-east-1, print the snapshot information for all instances.'
#query = 'In us-east-1, let me know which instance does not have a snapshot.'
#query = 'In us-east-1, how are my EC2 backups looking?'

## Amazon S3 queries
#query = 'In us-east-1, list all buckets.'
#query = 'List all buckets in all regions.'
#query = 'In us-west-2, get me the replication config on bucket named "Specify NAME".'
#query = 'In us-east-1, get me the lifecycle config on bucket named "Specify NAME".'
#query = 'In us-east-1, get me the versioning info on bucket named "Specify NAME".'

## Amazon RDS queries
#query = 'In us-east-1, list all RDS db instances.'
#query = 'In us-east-1, list all RDS db cluster automated backups.'
#query = 'In us-east-1, list all RDS db instance automated backups.'
query = 'In us-east-1, list all RDS db clusters.'

#### Examples of executor queries

## AWS Backup queries
#query = 'In the us-west-2 Region, create a plan to backup all the EC2 instances every 6 hours. Assume other values as required.'
#query = 'In us-west-2, backup all the EC2 instances with tag named "NAME" and value "TEST" every 4 hours. Assume other values as required.'
#query = 'Cancel legal hold with id "ID".'
#query = 'In us-east-1, delete backup vault named "NAME"'.
#query = 'In us-east-1, create a legal hold with title "TITLE" and description "DESC".'

## Amazon EC2 queries
#query = 'In us-east-1, delete snapshot with id "value".'
#query = 'In us-east-1, create snapshot for volume with id "ID".'

## Amazon S3 queries
#query = ''

## Amazon RDS queries
#query = 'In us-east-1, stop the RDS db cluster automated backup for cluster with resource id "value".'
#query = 'In us-east-1, stop the RDS db instance automated backup for instance with id "value".'
#query = 'In us-east-1, start the RDS db instance automated backups replication on the db instance with ARN "ARN".'
#query = 'In us-east-1, stop the RDS db instance automated backups replication on the db instance with ARN "ARN".'

# Invoke the supervisor Bedrock Agent
invoke_agent_response, invoke_agent_trace = invoke_agent(br_supervisor_agent_alias_id, br_supervisor_agent_id, bedrock_agt_rt_client, query)

# Print the response from the supervisor agent
logging.info(invoke_agent_response)

Run the following cell to print the trace of the actions that the sueprvisor agent took to process your query. This will also print the traces of the sub-agents and Knowledge Base interactions, if any.

In [ ]:
# Print the trace from the supervisor agent
logging.info(invoke_agent_trace)

## 4. Cleanup <a id='Cleanup'></a>

As a best practice, you should delete AWS resources that are no longer required.  This will help you avoid incurring unncessary costs.

<div class="alert alert-block alert-info">
<b>Note:</b> If you are running this notebook as part of a workshop session, by default, all resources will be cleaned up at the end of the session.
</div>

## 5. Conclusion <a id='Conclusion'></a>

We have now seen how to use Amazon Bedrock Agents to strategize, plan, and execute your data backup objectives on AWS through natural language interactions.

## 6. Frequently Asked Questions (FAQs) <a id='FAQs'></a>

**Q: What AWS services are used in this notebook?**

Amazon Bedrock (Agents, LLMs, and Knowledge Bases), Amazon OpenSearch Serverless, AWS Lambda, AWS Backup, Amazon EC2, Amazon S3, Amazon RDS, AWS Identity and Access Management (IAM), Amazon CloudWatch, and Amazon SageMaker Notebook instance (or) Amazon SageMaker Studio Notebook depending on what you use to run the notebook.

**Q: Where can I access the FAQs for Amazon Bedrock?**

Go [here](https://aws.amazon.com/bedrock/faqs/).

**Q: What models are supported by Amazon Bedrock?**

Go [here](https://docs.aws.amazon.com/bedrock/latest/userguide/models-supported.html).

**Q: Where can I find customer references for Amazon Bedrock?**

Go [here](https://aws.amazon.com/bedrock/testimonials/).

**Q: Where can I find pricing information for the AWS services used in this notebook?**

- Amazon Bedrock pricing - go [here](https://aws.amazon.com/bedrock/pricing/).
- Amazon OpenSearch Serverless pricing - go [here](https://aws.amazon.com/opensearch-service/pricing/).
- AWS Lambda pricing - go [here](https://aws.amazon.com/lambda/pricing/).
- AWS Backup pricing - go [here](https://aws.amazon.com/backup/pricing/).
- Amazon EC2 pricing - go [here](https://aws.amazon.com/ec2/pricing/).
- Amazon S3 pricing - go [here](https://aws.amazon.com/s3/pricing/).
- Amazon RDS pricing - go [here](https://aws.amazon.com/rds/pricing/).
- AWS Identity and Access Management (IAM) pricing - free.
- Amazon CloudWatch pricing - go [here](https://aws.amazon.com/cloudwatch/pricing/).
- Amazon SageMaker Notebook instance (or) Amazon SageMaker Studio Notebook pricing - go [here](https://aws.amazon.com/sagemaker/pricing/).